# Iterating through JSON

This is a short tutorial on the difference between using pandas apply to read in complex JSON vs iterating directly through the JSON itself. More than a 100x improvement is seen by avoiding the `apply` method.

In [1]:
import pandas as pd
import requests

In [2]:
url = 'https://eonet.sci.gsfc.nasa.gov/api/v2.1/events?limit=10000'
req = requests.get(url)
json = req.json()
events = json['events']
events[0]

{'id': 'EONET_4461',
 'title': 'Hunter Fire',
 'description': '',
 'link': 'https://eonet.sci.gsfc.nasa.gov/api/v2.1/events/EONET_4461',
 'categories': [{'id': 8, 'title': 'Wildfires'}],
 'sources': [{'id': 'InciWeb',
   'url': 'http://inciweb.nwcg.gov/incident/6645/'}],
 'geometries': [{'date': '2019-10-14T11:30:00Z',
   'type': 'Point',
   'coordinates': [-116.097, 40.897]}]}

In [3]:
def use_pandas_apply(json):
    df = pd.DataFrame(json)['events'].apply(pd.Series).set_index('id')
    
    df = df.categories.apply(pd.Series).stack().apply(pd.Series) \
           .reset_index(1, drop=True).title.rename('event').pipe(df.join)
    del df['categories']
    
    df = df.geometries.apply(pd.Series).stack().apply(pd.Series) \
           .join(df).reset_index(-1, drop=True)
    del df['geometries']
    
    df = df.coordinates.apply(pd.Series).rename(columns=dict(zip((0,1), "xy"))).join(df)
    del df['coordinates']
    
    df['date'] = pd.to_datetime(df.date)
    df = df[df.x.apply(lambda x: not isinstance(x, list))]
    return df

In [4]:
def iterate_through_json(events):
    event_dict = {'id': [], 'x': [], 'y': [], 'date': [], 'type': [], 'title': [],
                  'description': [], 'link': [], 'sources': [], 'event': []}
    for event in events:
        for geometry in event['geometries']:
            if len(geometry['coordinates']) == 2:
                event_dict['date'].append(geometry['date'])
                event_dict['x'].append(geometry['coordinates'][0])
                event_dict['y'].append(geometry['coordinates'][1])
                event_dict['type'].append(geometry['type'])
                event_dict['id'].append(event['id'])
                event_dict['title'].append(event['title'])
                event_dict['description'].append(event['description'])
                event_dict['link'].append(event['link'])
                event_dict['event'].append(event['categories'][0]['title'])
                event_dict['sources'].append(event['sources'])
                
    return pd.DataFrame(event_dict).set_index('id').sort_index()

In [5]:
use_pandas_apply(json).head()

,x,y,date,type,title,description,link,sources,event
id,,,,,,,,,
EONET_2632,161.36,56.6530,2016-09-18 00:00:00+00:00,Point,"Sheveluch Volcano, Russia",Sheveluch is one of the largest and most activ...,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'EO', 'url': 'https://earthobservatory...",Volcanoes
EONET_2654,-71.85,-15.7800,2016-11-07 00:00:00+00:00,Point,"Sabancaya Volcano, Peru",,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'EO', 'url': 'https://earthobservatory...",Volcanoes
EONET_2693,156.014,50.6860,2016-12-08 00:00:00+00:00,Point,"Ebeko Volcano, Russia",,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'SIVolcano', 'url': 'http://volcano.si...",Volcanoes
EONET_2734,-41.4727,-75.8853,2011-08-30 00:00:00+00:00,Point,Iceberg A23A,,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....",Sea and Lake Ice
EONET_2736,-108.526,-74.2006,2011-08-30 00:00:00+00:00,Point,Iceberg B22A,,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....",Sea and Lake Ice


In [6]:
iterate_through_json(events).head()

,x,y,date,type,title,description,link,sources,event
id,,,,,,,,,
EONET_2632,161.3600,56.6530,2016-09-18T00:00:00Z,Point,"Sheveluch Volcano, Russia",Sheveluch is one of the largest and most activ...,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'EO', 'url': 'https://earthobservatory...",Volcanoes
EONET_2654,-71.8500,-15.7800,2016-11-07T00:00:00Z,Point,"Sabancaya Volcano, Peru",,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'EO', 'url': 'https://earthobservatory...",Volcanoes
EONET_2693,156.0140,50.6860,2016-12-08T00:00:00Z,Point,"Ebeko Volcano, Russia",,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'SIVolcano', 'url': 'http://volcano.si...",Volcanoes
EONET_2734,-41.4727,-75.8853,2011-08-30T00:00:00Z,Point,Iceberg A23A,,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....",Sea and Lake Ice
EONET_2736,-108.5265,-74.2006,2011-08-30T00:00:00Z,Point,Iceberg B22A,,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....",Sea and Lake Ice


In [7]:
%timeit use_pandas_apply(json)

476 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit iterate_through_json(events)

3.53 ms ± 48.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
from pandas.io.json import json_normalize

In [10]:
def use_json_normalize(events):
    df1 = json_normalize(events, 'categories', ['id', 'title', 'description', 'link', 'sources'], 
                         record_prefix='categories_').set_index('id').drop(columns=['categories_id']) \
                                                     .rename(columns={'categories_title': 'event'})
    df2 = json_normalize(events, 'geometries', 'id').set_index('id')
    df2['x'] = df2['coordinates'].str[0]
    df2['y'] = df2['coordinates'].str[0]
    df2 = df2.drop(columns='coordinates')
    return df1.join(df2)

In [11]:
use_json_normalize(events).head()

,event,title,description,link,sources,date,type,x,y
id,,,,,,,,,
EONET_2632,Volcanoes,"Sheveluch Volcano, Russia",Sheveluch is one of the largest and most activ...,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'EO', 'url': 'https://earthobservatory...",2016-09-18T00:00:00Z,Point,161.36,161.36
EONET_2654,Volcanoes,"Sabancaya Volcano, Peru",,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'EO', 'url': 'https://earthobservatory...",2016-11-07T00:00:00Z,Point,-71.85,-71.85
EONET_2693,Volcanoes,"Ebeko Volcano, Russia",,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'SIVolcano', 'url': 'http://volcano.si...",2016-12-08T00:00:00Z,Point,156.014,156.014
EONET_2734,Sea and Lake Ice,Iceberg A23A,,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....",2011-08-30T00:00:00Z,Point,-41.4727,-41.4727
EONET_2736,Sea and Lake Ice,Iceberg B22A,,https://eonet.sci.gsfc.nasa.gov/api/v2.1/event...,"[{'id': 'BYU_ICE', 'url': 'http://www.scp.byu....",2011-08-30T00:00:00Z,Point,-108.526,-108.526


In [12]:
%timeit use_json_normalize(events)

19.5 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
